In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from scipy import special as sp
import scipy.integrate as integrate
from scipy import optimize
rng = np.random.default_rng()

#variaveis e constantes

fine_strc_const = 1/137          #[no units] fine-structure constant
mass_electron = 1.67*10**(-27)   #[Kg] massa do eletrao
h_bar = 6.626*10**(-34)/(2*pi)   #[J.s] constante de planck barra
spd_light = 3 * 10**8            #[m/s] velocidade da luz no vazio
lrtz_factor = 5                  #[no units] fator de lorentz
omega = 10**5                    #[1/s] frequencia do fotao
num_pon = 10**5                  #numero de pontos
n_bins = 10**2                   #numero de bins
n_gra = 4                        #numero de graficos

#calculo do primeiro conjunto de graficos

for i in range(n_gra):
    X_e = 0.5 * 10**i

    X_gamma = np.linspace(0.000001, X_e * 0.9999, num_pon)

    Epsilon = X_gamma / X_e
    
    #esta funcao de integracao foi criada para funcionar como a funcao a ser subtraida na expressao analitica. Contudo, aumenta em bastante
    #o tempo de processamento e, portanto, decidi nao implemta-la. Os gráficos continuam a ser bastante semelhantes aos pretendidos

    #def integration(x1):
     #   arr = np.array([integrate.quad(lambda x: sp.kv(1 / 3, x), value, np.Inf)[0] for value in x1])
      #  return arr

    def d2P_dtdX(x1):   #funcao analitica nao normalizada
        X_tilde = 2 * x1 / (3 * X_e * (1 - x1))
        return x1*X_e*fine_strc_const*mass_electron*spd_light**2/(np.sqrt(3)*pi*h_bar*lrtz_factor*X_e)*((1-x1+1/(1-x1))*sp.kv(2/3, X_tilde))

    N_d2P_dtdX = d2P_dtdX(Epsilon)

    integ = integrate.trapz(N_d2P_dtdX, Epsilon)  #integral da funcao analitica
    N_d2P_dtdX /= integ

    def d2P_dtdX_norm(x1):  #funcao analitica normalizada
        X_tilde = 2 * x1 / (3 * X_e * (1 - x1))
        return (x1 * X_e * fine_strc_const * mass_electron * spd_light ** 2 / (np.sqrt(3) * pi * h_bar * lrtz_factor * X_e) * ((1 - x1 + 1 / (1 - x1)) * sp.kv(2 / 3, X_tilde)))/integ

    plt.plot(Epsilon, N_d2P_dtdX)  #grafico da expressao analitica
    
    #inverse transform sampling method
    
    def dP_dt(x1):   #CDF
        return integrate.quad(lambda x: d2P_dtdX_norm(x), 0, x1)[0]
        
    #metodo para inverter a funcao
    
    def dP_dt_inverse_aux(y, x):
        return dP_dt(y) - x


    def dP_dt_inverse(x):
        return optimize.bisect(dP_dt_inverse_aux, 0.0000001, 0.999999, args=(x))


    dP_dt_ICDF = np.array([dP_dt_inverse(x) for x in Epsilon]) #IDCF

    n_0, bins_0, patches_0 = plt.hist(dP_dt_ICDF, n_bins, density=True, 
                                      facecolor='r', alpha=0.75)     #histograma da ICDF

    #rejection method
    
    i = 0

    max_func = np.max(N_d2P_dtdX)       #maximo da funcao analitica
    d2P_dtdX_Rej = np.zeros(num_pon)
    while (i < num_pon):
        x = rng.random()
        y = rng.random()
        if (y < d2P_dtdX_norm(x) / max_func):
            d2P_dtdX_Rej[i] = x              #distribuicao gerada pelo rejection method
            i += 1
    n_1, bins_1, patches_1 = plt.hist(d2P_dtdX_Rej, n_bins, density=True,
                                      facecolor='r', alpha=0.75)          #histograma da funcao gerada pelo rejection method
    
plt.show()

#calculo do segundo conjunto de graficos

for i in range(n_gra):
    X_gamma = 0.5 * 10 ** i
    X_e = np.linspace(0.000001, X_gamma * 0.9999, num_pon)

    Epsilon = X_e / X_gamma

    def d2P_dtdXe(x1):  #segunda funcao analitica nao normalizada
        X_tilde = 2 / (3 * X_gamma * (1 - x1) * x1)
        return fine_strc_const*mass_electron**2*spd_light**4/(np.sqrt(3)*pi*h_bar*omega*X_gamma)*((x1/(1-x1) + ((1-x1)/x1))*sp.kv(2/3, X_tilde))

    N_d2P_dtdXe = d2P_dtdXe(Epsilon)

    integ = integrate.trapz(N_d2P_dtdXe, Epsilon)  #integral da segunda funcao analitica
    N_d2P_dtdXe /= integ

    def d2P_dtdXe_norm(x1):   #segunda funcao analitica normalizada
        X_tilde = 2 / (3 * X_gamma * (1 - x1) * x1)
        return fine_strc_const * mass_electron ** 2 * spd_light ** 4 / (np.sqrt(3) * pi * h_bar * omega * X_gamma) * ((x1 / (1 - x1) + ((1 - x1) / x1)) * sp.kv(2 / 3,X_tilde))/integ

    plt.plot(Epsilon, N_d2P_dtdXe)
    
    #inverse transform sampling method
    
    def dP_dte(x1):     #segunda CDF
        return integrate.quad(lambda x: d2P_dtdXe_norm(x), 0, x1)[0]

    
    #metodo de invertar a funcao
    
    def dP_dte_inverse_aux(y, x):
        return dP_dte(y) - x

    

    def dP_dte_inverse(x):
        return optimize.bisect(dP_dte_inverse_aux, 0.0000001, 0.999999, args=(x))

    
    dP_dte_ICDF = np.array([dP_dte_inverse(x) for x in Epsilon])  #ICDF

    
    n_3, bins_3, patches_3 = plt.hist(dP_dte_ICDF, n_bins, density=True,
                                      facecolor='b', alpha=0.75)      #histograma da ICDF
    
    #rejection method
    
    i = 0

    max_func = np.max(N_d2P_dtdXe)         #maximo da funcao analitica
    d2P_dtdXe_Rej = np.zeros(num_pon)
    while (i < num_pon):
        x = rng.random()
        y = rng.random()
        if (y < d2P_dtdXe_norm(x) / max_func):
            d2P_dtdXe_Rej[i] = x               #funcao gerada pelo rejection method
            i += 1
    n_2, bins_2, patches_2 = plt.hist(d2P_dtdXe_Rej, n_bins, density=True,
                                      facecolor='b', alpha=0.75)         #histograma da funcao gerada pelo rejection method
    
plt.show()